In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import io
from tensorflow.python.keras import Model
from tensorflow.python.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Concatenate, Dropout, Lambda, Reshape, AveragePooling2D, Flatten, Dense, Add, UpSampling2D
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import glob
import os
import pickle
import segmentation_models as sm
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, CSVLogger, TensorBoard

from tensorflow.python import keras

from tensorflow.python.keras import layers

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# train and test dataset

In [ ]:
listTrainValid = glob.glob('/dir/Train*.tfrecords')
listTest = glob.glob('/dir/Test*.tfrecords')

In [ ]:
image_feature_description = {
  'image': tf.io.FixedLenFeature([], tf.string),
  'label': tf.io.FixedLenFeature([], tf.string),
  }

In [ ]:
def _parse_image_function(example_proto):
    features = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.io.decode_raw(features['image'], tf.float32)
    image.set_shape([16 * 224 * 224])
    image = tf.reshape(image, [224, 224,16])

    tensor1 = tf.convert_to_tensor((np.ones((224,224,16))*np.array([[0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,
                                                                   0.02,0.02,0.02,0.02, 0.03, 0.01]])).astype(np.float32))
    tensor2 = tf.convert_to_tensor((np.ones((224,224,16))*np.array([[0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0]])).astype(np.float32))

    image = tf.multiply(image, tensor1)
    image = tf.add(image, tensor2)

    label = tf.io.decode_raw(features['label'], tf.float32)
    label.set_shape([1 * 224 * 224])
    label = tf.reshape(label, [ 224, 224,1])

    return image[:,:,10:15], label

In [ ]:
def read_dataset(epochs, batch_size, filenames):

    dataset = tf.data.TFRecordDataset(filenames)

    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.string),
    }

    dataset = dataset.map(_parse_image_function, num_parallel_calls=10)
    dataset = dataset.prefetch(64)
    dataset = dataset.repeat(epochs)
    dataset = dataset.shuffle(buffer_size= batch_size)
    dataset = dataset.batch(batch_size, drop_remainder=True)

    return dataset

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()
devices = tf.config.experimental.list_physical_devices("GPU")
print(devices)

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1","/gpu:2", "/gpu:3"])

# Deep Model

In [ ]:
ListBackbone = ['inceptionv3','vgg19','resnet18','mobilenetv2']

In [ ]:
for backbone in ListBackbone:

    for i in range(5):
        #i = j + 5
        listTrain, listValidation = train_test_split(listTrainValid , test_size=0.2, random_state=i)

        train_dataset = read_dataset(1, 64, listTrain)
        validation_dataset  = read_dataset(1, 64, listTrain)

        mirrored_strategy = tf.distribute.MirroredStrategy()
        devices = tf.config.experimental.list_physical_devices("GPU")
        mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1","/gpu:2", "/gpu:3"])

        sm.set_framework('tf.keras')
        sm.framework()

        with mirrored_strategy.scope():

            model1 = sm.Unet(backbone_name=backbone, input_shape=(224, 224, 5), activation='linear', encoder_weights=None)

            print(model1.summary())

            model1.compile( loss='mean_squared_error', optimizer='adam')

            checkpoint = keras.callbacks.ModelCheckpoint(filepath =  '/dir/Models/Unet_' + backbone + '_S1_NL_' + str(i) ,
                                                 monitor = 'val_loss',
                                                 verbose=1,
                                                 #save_weights_only=True,
                                                 save_best_only = True)

            csv_logger = keras.callbacks.CSVLogger( '/dir/Models/Unet_' + backbone + '_S1_NL_' +  str(i) + '_history.csv', append=True)

            history = model1.fit(x=train_dataset , validation_data=validation_dataset, epochs=500, batch_size=64, callbacks =[checkpoint, csv_logger])


for backbone in ListBackbone:

    for i in range(5):
        #i = j + 5
        listTrain, listValidation = train_test_split(listTrainValid , test_size=0.2, random_state=i)

        train_dataset = read_dataset(1, 64, listTrain)
        validation_dataset  = read_dataset(1, 64, listTrain)

        mirrored_strategy = tf.distribute.MirroredStrategy()
        devices = tf.config.experimental.list_physical_devices("GPU")
        mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1","/gpu:2", "/gpu:3"])

        sm.set_framework('tf.keras')
        sm.framework()

        with mirrored_strategy.scope():

            model1 = sm.Linknet(backbone_name=backbone, input_shape=(224, 224, 5), activation='linear', encoder_weights=None)

            print(model1.summary())

            model1.compile( loss='mean_squared_error', optimizer='adam')

            checkpoint = keras.callbacks.ModelCheckpoint(filepath =  '/dir/Models/Linknet_' + backbone + '_S1_NL_' + str(i) ,
                                                 monitor = 'val_loss',
                                                 verbose=1,
                                                 save_best_only = True)

            csv_logger = keras.callbacks.CSVLogger( '/dir/Models/Linknet_' + backbone + '_S1_NL_' +  str(i) + '_history.csv', append=True)

            history = model1.fit(x=train_dataset , validation_data=validation_dataset, epochs=500, batch_size=64, callbacks =[checkpoint, csv_logger])

